In [ ]:
import pandas as pd
import wrangleutils as wr
import glob as glob
import os
from collections import Counter
%matplotlib inline

In [ ]:
#Define the path to scored data sets
path="\\path"
#Define the name of your current scored set 
##where '.' is the working directory (i.e. the directory you launched the notebook from)
#Creates a list of all .csvs in  that file source
csv_list=glob.glob(os.path.realpath('.')+ path +'\\*.csv')
#Concats all CSVs in folder
df=pd.concat(list(map(pd.read_csv, csv_list)))

In [ ]:
#Look at the data types in this data set. Creates a meta data data frame
meta_data=pd.DataFrame(df.dtypes)
meta_data['name']=meta_data.index


In [ ]:
#Record the original dimensions of the dataset before messing around with it 
original_dim=df.shape
print("There are " + str(original_dim[1]) + " columns" + " and "  + str(original_dim[0])+ " rows in the original data")

In [ ]:
#Create a list of categorical variables
cat_list= df.select_dtypes(include=['object'])

#Delete any categorical vars from the list you
#know you arent interested in
#del cat_list[('variable')]

#Create a list of floats
float_list= list(df.select_dtypes(include=['float']))
#Create a list of ints
int_list= list(df.select_dtypes(include=['int64']))

In [ ]:
#Checks wheher any float columns have a max of 0 and are useless
max_zero=pd.DataFrame(df[float_list].columns[df[float_list].aggregate(max)==0])
max_zero_list=list(df[float_list].columns[df[float_list].aggregate(max)==0])
print("Warning - the following numeric columns have a maximum of 0:")
max_zero


In [ ]:
#Drops the max zero list
df.drop(max_zero_list, axis=1, inplace=True)
#Drops the max zero list fom the float list for later analysis 
float_list =  [x for x in float_list if x not in max_zero_list]
#Find the shape of the new data frame with the max 0 columns dropped 
new_dim=df.shape
print( "There were "  + str(original_dim[1]) + " columns and now are " + str(new_dim[1]) + " columns")

In [ ]:
#Drops the max zero list
df.drop(max_zero_list, axis=1, inplace=True)
#Drops the max zero list fom the float list for later analysis 
float_list =  [x for x in float_list if x not in max_zero_list]
#Find the shape of the new data frame with the max 0 columns dropped 
new_dim=df.shape
print( "There were "  + str(original_dim[1]) + " columns and now are " + str(new_dim[1]) + " columns")

In [ ]:
#Function to do frequency anlysis
def freq_iter(df, cols):
    dflist = []
    for i in cols:
        series = df[i]
        #Count the discrete values
        type_count = [[k,v] for k,v in Counter(series).items()]
        # Find the count of all discrete elements 
        j=pd.DataFrame.from_records(type_count, 
                                    index=range(len(type_count)), 
                                    columns=['value','frequency'])
        j['percent'] = j.frequency/j.frequency.sum() 
        j['cumulative_frequency'] = j.frequency.cumsum()
        j['cumulative_percent'] = j.cumulative_frequency/j.frequency.sum()
        j.columns = [series.name + '_' + i for i in j.columns]
        dflist.append(j)
    return dflist

In [ ]:
#Create a list with the frequency analysis function and displays the contents 
freq_iter_list = freq_iter(df, cat_list)
for p in freq_iter_list: 
     display (p.iloc[:,[0,1,2]])

In [ ]:
#Displays the variance if all the float columns
ratio_of_std_to_mean=pd.DataFrame(df[float_list].std()) /pd.DataFrame(df[float_list].mean())
#####
#### Keep working on this part 

In [ ]:
#Calculates % missing by columns
missing=pd.DataFrame(100* df.isna().sum() /len(df))
missing['pct_missing']=missing[0]
#Optional missing threshold
threshold=20
missing_above_threshold= missing[missing['pct_missing']>threshold]
missing_above_threshold=pd.DataFrame(missing_above_threshold.iloc[:,1])

#Shows all missing regardles sof threshold 
missing_above_zero= missing[missing['pct_missing']>0]
missing_above_zero=pd.DataFrame(missing_above_zero.iloc[:,1])
print("The following fields have missing values:")
missing_above_zero

In [ ]:
print("Warning the following columns are missing more than 20 % of their values. Suggest either you bin, drop, or impute based on domain knowledge :")
missing_above_threshold

In [ ]:
#This list has to be compiled manually, as far as I can tell.
#I think that it is more effeicient to create a list of vars that should NOT
#be coalesced to 0, then subtract that out to create the coalesce 0 list of fields

bin_list = ['var1', 'var2', 'vark'] 





In [ ]:
#Converts the values of the missing above 0 dataframe to a list 
missing_above_zero_list = missing_above_zero.index.tolist()
#Creates a list of variables (that arent in the binned list)
#That have missing values that you want to replace with 0 
zero_replace_list= [x for x in missing_above_zero_list if x not in bin_list]

#Alters the base data frame by replacing the NaN with 0 in the specified columns above
df[zero_replace_list]=df[zero_replace_list].fillna(0)
print("Warning: the NaN with 0 replacement has been applied to the base dataframe")

In [ ]:
#Automatically converts variables in the bin list to bins